# Artifact - Queries

** For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md **

1 - Iroha Blockchain Testnet

1.1 -  General modules and definitions for authentication, connection, status and error tracing.

In [3]:
import os
import sys
import binascii
from grpc import RpcError, StatusCode
import inspect
from iroha import Iroha, IrohaGrpc, IrohaCrypto
from iroha.primitive_pb2 import can_call_engine
from functools import wraps
import json

# Load configuration from config.json file
config_path = "config.json"  # Update this path as needed
with open(config_path, "r") as f:
    config = json.load(f)

IROHA_HOST_ADDR = config["IROHA_HOST_ADDR"]
IROHA_PORT = config["IROHA_PORT"]
ADMIN_ACCOUNT_ID = config["ADMIN_ACCOUNT_ID"]
ADMIN_PRIVATE_KEY = config["ADMIN_PRIVATE_KEY"]

iroha = Iroha(ADMIN_ACCOUNT_ID)
net = IrohaGrpc(f'{IROHA_HOST_ADDR}:{IROHA_PORT}')

def trace(func):
    @wraps(func)
    def tracer(*args, **kwargs):
        name = func.__name__
        stack_size = int(len(inspect.stack(0)) / 2)
        indent = stack_size * '\t'
        print(f'{indent} > Entering "{name}": args: {args}')
        result = func(*args, **kwargs)
        print(f'{indent} < Leaving "{name}"')
        return result
    return tracer

@trace
def send_transaction_and_print_status(transaction):
    hex_hash = binascii.hexlify(IrohaCrypto.hash(transaction))
    creator_id = transaction.payload.reduced_payload.creator_account_id
    commands = get_commands_from_tx(transaction)
    print(f'Transaction "{commands}", hash = {hex_hash}, creator = {creator_id}')
    net.send_tx(transaction)
    handle_transaction_errors(transaction)

def handle_transaction_errors(transaction):
    for i, status in enumerate(net.tx_status_stream(transaction)):
        status_name, status_code, error_code = status
        print(f"{i}: status_name={status_name}, status_code={status_code}, error_code={error_code}")
        if status_name in ('STATEFUL_VALIDATION_FAILED', 'STATELESS_VALIDATION_FAILED', 'REJECTED'):
            handle_error(status_name, error_code, transaction)

def handle_error(status_name, error_code, transaction):
    error_messages = {
        'STATEFUL_VALIDATION_FAILED': 'Stateful validation failed',
        'STATELESS_VALIDATION_FAILED': 'Stateless validation failed',
        'REJECTED': 'Transaction rejected'
    }
    error_message = error_messages.get(status_name, 'Unknown error') + f': {error_code}'
    raise RuntimeError(f"{status_name} failed on tx: {transaction} due to reason {error_code}: {error_message}")

def get_commands_from_tx(transaction):
    commands_from_tx = []
    for command in transaction.payload.reduced_payload.__getattribute__("commands"):
        listed_fields = command.ListFields()
        commands_from_tx.append(listed_fields[0][0].name)
    return commands_from_tx

1.2 - Queries -Executing a sequence of queries into the iroha blockchain as the admin user.

1.2.1 - Get Account Transactions

In [4]:
#Query - GetAccountTransactions
query = iroha.query('GetAccountTransactions', account_id=ADMIN_ACCOUNT_ID, page_size=3)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response
print(data)

query_hash: "0614692792ac6796ad34cdbde0c627117d1d9c38a1247ae1bf17bc713b5792bb"
transactions_page_response {
  transactions {
    payload {
      reduced_payload {
        commands {
          call_engine {
            caller: "admin@test"
            input: "608060405234801561001057600080fd5b5073a6abc17819738299b3b2c1ce46d55c74f04e290c6000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555061096b806100746000396000f3fe608060405234801561001057600080fd5b50600436106100415760003560e01c80634518f6b314610046578063bc53c0c414610076578063d4e804ab146100a6575b600080fd5b610060600480360381019061005b9190610486565b6100c4565b60405161006d91906106ad565b60405180910390f35b610090600480360381019061008b91906104c7565b610230565b60405161009d91906106ad565b60405180910390f35b6100ae6103fa565b6040516100bb9190610692565b60405180910390f35b60606000826040516024016100d991906106cf565b6040516020818303038152906040527f4518f6b3000000000000000000000000000000000

In [ ]:
1.2.2 - GetRoles

In [5]:
#Query - GetRoles
query = iroha.query('GetRoles')
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response
print(data)

roles_response {
  roles: "admin"
  roles: "user"
  roles: "money_creator"
}
query_hash: "be32e7c189f7c8efdf3fcc6cfecd138d3185c88e9727f16a9cb0b573138d8445"



In [ ]:
1.2.3 - GetRolePermissions - admin

In [4]:
#Query - GetRolePermissions
ROLE_ID="admin"
query = iroha.query('GetRolePermissions',role_id=ROLE_ID)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response
print(ROLE_ID, data)

admin role_permissions_response {
  permissions: root
}
query_hash: "f4008cae9ffc9e7625f814dc19318db6422a6c5f0c269b2098725e3a0f9adf37"



In [ ]:
1.2.4 - GetRolePermissions - user

In [ ]:
#Query - GetRolePermissions
ROLE_ID="user"
query = iroha.query('GetRolePermissions',role_id=ROLE_ID)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response
print(ROLE_ID, data)

In [ ]:
1.2.5 - GetRolePermissions - money_creator

In [ ]:
#Query - GetRolePermissions
ROLE_ID="money_creator"
query = iroha.query('GetRolePermissions',role_id=ROLE_ID)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response
print(ROLE_ID, data)

In [ ]:
1.2.6 - GetRolePermissions - first_role

In [3]:
ROLE_ID="first_role"
query = iroha.query('GetRolePermissions',role_id=ROLE_ID)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response
print(ROLE_ID, data)

first_role error_response {
  reason: NO_ROLES
  message: "no role with such name in account with such id: {first_role, admin@test}"
}
query_hash: "d384263584765e1aff3f9457bc049b8519a7893b5575e034f2973b4e979b9125"



In [ ]:
1.2.7 - GetAccountDetail - admin@test

In [4]:
# #Query - GetAccountDetail
user = 'admin@test'
query = iroha.query('GetAccountDetail',account_id=user)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response.account_detail_response
print(f'Account id = {user}, details = {data.detail}')

Account id = admin@test, details = { "admin@test" : { "super" : "019050919050565b600081519050919050565b600081519050919050565b600082825260208201905092915050565b600081905092915050565b60008" } }


In [ ]:
1.2.8 - GetAccountDetail - ecstatic_dubinsky@test

In [7]:
#Query - GetAccountDetail
user = 'happy_raman@test'
query = iroha.query('GetAccountDetail',account_id=user)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response.account_detail_response
print(f'Account id = {user}, details = {data.detail}')

Account id = happy_raman@test, details = 
